In [11]:
import os, pathlib, uuid, sys, asyncio
from dotenv import load_dotenv
load_dotenv()
# from langgraph.prebuilt import create_react_agent
from langchain.agents.react.agent import create_react_agent
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_core.tools import tool
from langchain.agents import AgentExecutor
# from langgraph.checkpoint.memory import MemorySaver
# from langchain.memory import ConversationBufferMemory
# from langchain_core.messages import HumanMessage
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

In [2]:
ollama_url = os.getenv("ollama_url")
ollama_model = os.getenv("ollama_model")

In [3]:
llm = ChatOpenAI(
    model=ollama_model,
    base_url=ollama_url+"v1",
    api_key="ollama",
    temperature=0.2,
)

In [4]:
BASE_DIR = pathlib.Path.cwd()
BASE_DIR

WindowsPath('c:/FinalProjectAIWorkspace/notebooks')

In [5]:
SERVER_PATH = (BASE_DIR / ".." / "src" / "agents" / "mcp" / "server.py").resolve()
SERVER_PATH

WindowsPath('C:/FinalProjectAIWorkspace/src/agents/mcp/server.py')

In [6]:
from langchain_mcp_adapters.client import MultiServerMCPClient

In [7]:
str(SERVER_PATH).replace("\\", "/")

'C:/FinalProjectAIWorkspace/src/agents/mcp/server.py'

In [13]:
SERVER_PATH.parent

WindowsPath('C:/FinalProjectAIWorkspace/src/agents/mcp')

In [8]:
client = MultiServerMCPClient(
    {
        "RAG_and_weather": {
            "transport": "stdio",  # Local subprocess communication
            "command": "uv",
            # Absolute path to your math_server.py file
            "args": ["run", "python", str(SERVER_PATH).replace("\\", "/")],
        }
    }
)

In [12]:
tools = asyncio.run(client.get_tools())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [10]:
tools

<coroutine object MultiServerMCPClient.get_tools at 0x00000183F7455930>

In [ ]:
@tool
def ping(host: str) -> str:
    """Return 'pong' pretending to ping a host."""
    # 진짜 핑을 치려면 subprocess로 ping 실행하거나, requests로 healthcheck 등 구현
    return f"pong:{host}"

In [ ]:
@tool
def get_user_age(name: str) -> str:
    """Use this tool to find the user's age."""
    # This is a placeholder for the actual implementation
    if "bob" in name.lower():
        return "42 years old"
    return "41 years old"

In [ ]:
prompt = hub.pull("hwchase17/react-chat")

In [ ]:
agent = create_react_agent(llm, [ping, get_user_age], prompt=prompt)
# agent = create_react_agent(llm, [ping, get_user_age], checkpointer=memory)

In [ ]:
# memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [ ]:
# agent_executor = AgentExecutor(agent=agent, tools=[ping, get_user_age], verbose=True, memory=memory)
agent_executor = AgentExecutor(agent=agent, tools=[ping, get_user_age], verbose=True)

In [ ]:
# 세션 저장소 (간단한 in-memory dict 예시)
store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

agent_with_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [ ]:
# 세션 아이디는 네가 정한다
cfg = {"configurable": {"session_id": "user-123"}}

In [ ]:
res = agent_with_history.invoke({"input": "hi! I'm bob. What is my age?"}, config=cfg)
print(res)

In [ ]:
res = agent_with_history.invoke({"input": "do you remember my name?"}, config=cfg)
print(res)